In [90]:
import qrcode
import pandas as pd
from PIL import Image
import io    
import re
import pymongo
from fpdf import FPDF
import random
from datetime import datetime,timedelta,time

In [91]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [92]:
db = client.aadhar

In [93]:
records = db.card

In [94]:
def aadhar():
  r=random.randint(1000,9999)
  s=random.randint(1000,9999)
  t=random.randint(1000,9999)
  u=random.randint(1000,9999)
  aadhar = str(r)+"-"+str(s)+"-"+str(t)+"-"+str(u)
  return aadhar

In [95]:
def pic():
    im = Image.open("dhoni.jpg")
    image_bytes = io.BytesIO()
    im.save(image_bytes, format='JPEG')

    prof_pic = {
    "image":image_bytes.getvalue()
    }
    return prof_pic
photo = {"Photo":pic()}

In [96]:
def date():
    d = input("Enter your date of birth in DD/MM/YYYY format: ")
    date_format= "^([0]?[1-9]|[1|2][0-9]|[3][0|1])[./-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$"
    if re.match(date_format, d):
        dob = {"Date of Birth": d}
        return dob
    else:
        print("Enter valid date")   

In [97]:
def address():
    Plot_Number= input("Enter Plot Number: ")
    Street= input("Enter Street Name: ")
    City= input("Enter City: ")
    State= input("Enter State: ")
    Pincode= int(input("Enter Pincode Number: "))
    
    address={
              "No,street":Plot_Number +','+Street+',',
              "City":City+',',
              "State":State+',',
              "Pincode":City+'-'+str(Pincode)+'.'
          }
    return address

In [98]:
def edu():
    print(''' Select one option : 
                1. NA
                2. 10th
                3. 12th
                4. Bachelors
                5. Masters
                6. Doctorate''')
    n = int(input("Enter the option: "))
    if n == 1:
        e = {"Education": "NA", "Stream":"NA"}
        return e
    elif n == 2:
        e = {"Education": "10th standard", "Stream":"NA"}
        return e
    elif n == 3:
        e = {"Education": "12th standard","Stream":"NA"}
        return e
    elif n == 4:
        ug = input("Enter Bachelors degree (Eg:B.E,Btech): ")
        stream = input("Enter Stream (Eg:Mechanical, English): ")
        
        e={"Education": ug,"Stream": stream}
        return e
    elif n == 5:
        pg = input("Enter Masters degree (Eg:M.E,Mtech): ")
        stream = input("Enter Stream (Eg:Mechanical, English): ")
        
        e={"Education": pg, "Stream": stream}
        return e
    elif n == 6:
        stream = input("Enter Stream (Eg:Mechanical, English): ")
        
        e={"Education": "PhD", "Stream": stream}
        return e
    else:
        print("invalid input")


In [99]:
def email():
    email_id = input("Enter valid Email address: ")
    k = "[\w\.-]+@[\w\.-]+\.[\w]+"
    if re.fullmatch(k, email_id):
        return email_id
    else:
        print("Your Email is not valid")

In [100]:
def created():
    now = datetime.now()
    date= datetime.strftime(now, '%d/%m/%y')
    return date

In [101]:
def renewal():
    now = datetime.now()+timedelta(days=730)
    renew = datetime.strftime(now,'%d/%m/%y')
    return renew

In [102]:
def aadhar_detail():
    a={
        "Aadhar_Id": aadhar(),
        "Name": input("Enter your Name: "),
        "photo":pic(),
        "Gender": input("Enter you gender (M/F/T): "),
        "Date of Birth": date(),
        "Address": address(),
        "Education": edu(),
        "Email": email(),
        "Created at": created(),
        "Renewal Date": renewal()
    }

    records.insert_one(a)
    return a

In [103]:
def update_contact():
    aadhar = input("Enter your aadhar number: ")
    m = input('''Select one option to update: 
              1. Name
              2. Email_id
              3. Address
              4. profile_pic
              5. Education Details \n''')
    if m<6:
        if m==1:
            name = input("Enter full name: ")
            find={"Aadhar_Id":aadhar}
            update={"$set":{"Name":name}}
            records.update_one(find, update)
            print("Updated")
        elif m==2:
            gender = input("Enter your Gender: ")
            find={"Aadhar_Id":aadhar}
            update={"$set":{"Gender":gender}}
            records.update_one(find,update)
            print("Updated")
        elif m==3:
            find={"Aadhar_Id":aadhar}
            update={"$set":{"Address":address()}}
            records.update_one(find, update)
            print("Updated")
        elif m == 4:
            find={"Aadhar_Id": aadhar}
            update={"$set":{"photo":pic()}}
            records.update_one(find, update)
            print("Updated")
        elif m == 5:
            find={"Aadhar_Id":aadhar}
            update={"$set":{"Education":edu()}}
            records.update_one(find, update)
            print("Updated")
    else:
        print("Select Valid option")

        

In [140]:
from tabulate import tabulate
def card():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=5)

    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L,
                      box_size=2, border=4)

    aadhar=input("Enter the Aadhar number: ")
    c = {"Aadhar_Id":aadhar}
    details = records.find(c,{'_id':0, 'Aadhar_Id':1, 
                             "Name":1, "Date of Birth":1,
                             "Gender":1,"Education.Education":1, "Education.Stream":1,
                             "Address.No,street":1,"Address.City":1, "Address.State":1, "Address.Pincode":1,"Email":1,"Renewal Date":1})

    for i in details:
        pdf.set_font("Arial", size=14)
        aadhar= ( "                        {}   "   .format(i['Aadhar_Id']))
        pdf.cell(180,15, aadhar,ln=1,align='C') 
        nam=         ( "           Name                    :        {}           " .format(i["Name"]))        
        dob=         ( "             DOB                     :       {}           "  .format(i['Date of Birth'])  )
        Gender=      ( "           Gender                  :       {}           "  .format(i["Gender"]))
        education=   ( "           Education Details       :         "          )
        education1 = ( "                  Education        :       {}         " .format(i['Education']['Education']))
        stream =     ( "                   Stream          :       {}         " .format(i['Education']['Stream']))
        address=     ( "           Address                 :                   "  )
        street=      ( "                           Street  :    {}     "  .format(i["Address"]['No,street']))
        city=        ( "                City            :       {}          "  .format(i["Address"]['City']))
        state=       ( "                   State           :       {}          "  .format(i["Address"]['State']))
        pincode=     ( "                   Pincode         :       {}          "  .format(i["Address"]['Pincode']))
        email=       ( "           Email-id                :       {}          "  .format(i["Email"])) 
        renewaldate= ( "           renewal_date            :       {}          "  .format(i["Renewal Date"]))
    
    pdf.set_font("Arial",size=10)
    pdf.set_fill_color(255, 255, 255)
    pdf.cell(200,10, nam,ln=2,align='C')
    pdf.cell(180,10, dob,ln=3,align='C')
    pdf.cell(182,10, Gender,ln=4,align='C')
    pdf.cell(158,10, education,ln=5,align='C')
    pdf.cell(175,10, education1,ln=6,align='C')
    pdf.cell(175,10, stream,ln=7,align='C')
    pdf.cell(173,10, address,ln=8,align='C')
    pdf.cell(180,10, street,ln=9,align='C')
    pdf.cell(187,10, city,ln=10,align='C')
    pdf.cell(196,10, pincode,ln=11,align='C')
    pdf.cell(211,10, email,ln=12,align='C')
    pdf.cell(196,19, renewaldate,ln=13,align='C')
    
    retival= records.find_one(c)  
    pil_im = Image.open(io.BytesIO(retival['photo']['image']))
    img = pil_im.crop()
    image=img.save("qr.jpg") 
    pdf.image("qr.jpg",20,30,30,40)
    
    qr.add_data(qr)    
    qr.make(fit = True)    
    qr_img = obj_qr.make_image(fill_color = "black", back_color = "white")    
    qr_img.save("aadharqr.png")  
    pdf.image("aadharqr.png",170,40,35,35)
    pdf.output("Dhoni.pdf")
    



In [141]:
op= int(input(''' Select an option 
             1. Create Aadhar
             2. Edit Aadhar
             3. Download Aadhar \n'''))
if op == 1:
    aadhar_detail()
elif op == 2:
    update_contact()
elif op == 3:
    card()
else:
    print("Enter valid input")


 Select an option 
             1. Create Aadhar
             2. Edit Aadhar
             3. Download Aadhar 
3
Enter the Aadhar number: 9732-1503-7098-1461
